In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.testing import assert_frame_equal
import matplotlib.pyplot as plt
import seaborn as sns
p1 = Path.cwd() / 'back_data'

In [2]:
flights = pd.read_csv(p1 / 'flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [4]:
# groupby() 메서드의 호출 결과는 groupby 객체 -> 메서드를 체인시킬 때 결과 출력
(flights
# 그룹화 열('AIRLINE'), 집계열('ARR_DELAY' -> 리스트 전달로 DataFrame 가능), 집계함수(agg('mean')) 확인
.groupby('AIRLINE')[['ARR_DELAY']]
# agg() 메서드와 함께 집계 함수를 사용하지 않으면 예외 발생
.agg('mean')
.head()
)

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


In [6]:
(flights
# 요일별로 모든 항공사의(그룹화 열) 취소, 우회한 항공편(집계열)
.groupby(['AIRLINE', 'WEEKDAY'])[['CANCELLED', 'DIVERTED']]
# 그 수와 비율(집계 함수) -> CANCELLED, DIVERTED 열은 1과 0으로 정리되어있기 때문
.agg(['sum', 'mean'])
.head()
)

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786

In [8]:
(flights
# 여러 그룹화열이 있을 경우 계층적 index 생성
.groupby(['ORG_AIR', 'DEST_AIR'])
# 여러 집계열이 있을 경우 계층적 columns 생성
# 집계열과 집계함수를 딕셔너리 형태로 정리 가능
.agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
.head()
)

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

In [11]:
# pd.NamedAgg() 함수를 통해 비계층 열 생성 가능 -> columns의 level이 한 단계
(flights
.groupby(['ORG_AIR', 'DEST_AIR'])
# agg() 메서드 내 {column_name}=pd.NamedAgg() 활용 -> column과 aggfunc 파라미터로 진행
.agg(sum_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='sum'),
mean_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='mean'),
# size 집계 함수는 그룹당 총 행의 개수 반환 / count 집계 함수는 그룹당 비결측치 개수를 반환
size_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='size'),
mean_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='mean'),
var_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='var'))
.head()
)

sum_cancelled  mean_cancelled  size_cancelled  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0             0.0              31   
        ABQ                   0             0.0              16   
        ABY                   0             0.0              19   
        ACY                   0             0.0               6   
        AEX                   0             0.0              40   

                  mean_air_time  var_air_time  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692

In [12]:
res = (flights
.groupby(['ORG_AIR', 'DEST_AIR'])
.agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
.head()
)
# res의 columns를 columns 속성의 to_flat_index() 메서드 및 list comprehension을 활용하여 펼치기 가능
res.columns = ['_'.join(col) for col in res.columns.to_flat_index()]
res.head()

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0             0.0              31   
        ABQ                   0             0.0              16   
        ABY                   0             0.0              19   
        ACY                   0             0.0               6   
        AEX                   0             0.0              40   

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692

In [13]:
# 열을 펼치는 함수를 만들어 pipe() 메서드 체인도 가능
def flatten_cols(df):
    df.columns = ['_'.join(col) for col in df.columns.to_flat_index()]
    return df
(flights
.groupby(['ORG_AIR', 'DEST_AIR'])
.agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
.pipe(flatten_cols)
.head()
)

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0             0.0              31   
        ABQ                   0             0.0              16   
        ABY                   0             0.0              19   
        ACY                   0             0.0               6   
        AEX                   0             0.0              40   

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692

In [15]:
(flights
# 그룹화열 중 하나가 'category' 형식일 경우 그냥 groupby를 수행하면 카티션 곱 발생
.assign(ORG_AIR=flights.ORG_AIR.astype('category'))
# groupby() 메서드에 observed=True 인자 전달하여 카티션 곱 제거
.groupby(['ORG_AIR', 'DEST_AIR'], observed=True)
.agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
.head()
)

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
LAX     ABQ              1  0.018182   55   89.259259   29.403215
        ANC              0  0.000000    7  307.428571   78.952381
        ASE              1  0.038462   26  102.920000  102.243333
        ATL              0  0.000000  174  224.201149  127.155837
        AUS              0  0.000000   80  150.537500   57.897310

In [18]:
airline_info = (flights
.groupby(['AIRLINE', 'WEEKDAY'])
# 집계열이 최상위 level, 집계함수가 하위 level로 작동
.agg({'DIST':['sum', 'mean'], 'ARR_DELAY':['min', 'max']})
.astype('int')
)
airline_info.head()

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732

In [19]:
# columns 속성의 get_level_values() 메서드 활용하여 희망하는 열 출력
airline_info.columns.get_level_values(0), airline_info.columns.get_level_values(1)

(Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object'),
 Index(['sum', 'mean', 'min', 'max'], dtype='object'))

In [20]:
# 기본적으로 groupby 작업은 모든 그룹화 열을 인덱스로
(flights
# as_index=False 인자 전달하여 인덱스 미지정 가능(groupby 작업 후 reset_index()를 체인시키는 것과 같은 결과)
# sort 파라미터도 활용 가능하며, 기본값은 True -> 데이터가 나타나는 순서와 동일하게 유지하려면 sort=False 인자 전달
.groupby(['AIRLINE'], as_index=False)['DIST']
.agg('mean')
.round(0)
.head()
)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0


In [21]:
college = pd.read_csv(p1 / 'college.csv')

In [26]:
# pandas나 numpy에 없는 사용자 정의 함수를 통해 집계 가능
# 한 기관의 평균에서의 최대 표준편차를 구하는 함수 작성
def max_deviation(ser):
    std_score = (ser - ser.mean()) / ser.std()
    # Series에 max() 메서드를 연결시켜 단일 스칼라 값 반환
    return std_score.abs().max()
# pandas는 기본적으로 함수 이름을 반환된 열의 이름으로 사용 -> __name__ 함수 속성을 사용해 수정 가능
max_deviation.__name__ = 'Max Deviation'
(college
.groupby('STABBR')['UGDS']
# 내장된 함수와 사용자 정의 함수 함께 사용 가능
.agg([max_deviation, 'count'])
.round(1)
.head()
)

,Max Deviation,count
STABBR,,
AK,2.6,10
AL,5.8,89
AR,6.3,82
AS,NaN,1
AZ,9.9,126


In [28]:
# groupby의 agg() 메서드는 agg(func, *args, **kwargs)로 구성 -> *args, **kwargs 파라미터 사용해 인자를 함수에 전달 가능
def pct_between(ser, low=1000, high=3000):
    # low이상 high이하인 경우 True, 아니면 False -> mead() 메서드를 통해 평균을 내면서 단일 스칼라 반환
    # between() 메서드는 Series에서만 가능
    return ser.between(low, high).mean() * 100
(college
.groupby(['STABBR', 'RELAFFIL'])['UGDS']
# agg() 메서드에 함수 및 *args 전달(함수들을 전달할 때는 리스트를 활용하지만 *args, **kwargs 전달 시에는 리스트 미사용)
.agg(pct_between, 1000, 10000)
.round(1)
.head()
)

STABBR  RELAFFIL
AK      0           42.9
        1            0.0
AL      0           45.8
        1           37.5
AR      0           39.7
Name: UGDS, dtype: float64

In [29]:
# 사용자 정의 함수의 파라미터를 사용하여 이름을 지정하고 싶은 경우, 함수 내에서 __name__ 속성 사용 필요
def between_n_m(n, m):
    def wrapper(ser):
        return pct_between(ser, n, m)
    wrapper.__name__ = f'between_{n}_{m}'
    return wrapper